# First Agent
We'll use smolagents, a library that provides a freamework for developing your agents with ease. not suitable for production grade

## What does this agent do?

In [13]:
from smolagents import CodeAgent, tool, InferenceClientModel
import os
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [56]:
# LLM model
model = InferenceClientModel(
    max_tokens = 2096,
    model_id = 'Qwen/Qwen2.5-Coder-32B-Instruct',
)

In [57]:
# simply use the model to generate response, not an agent here
agent = CodeAgent(model= model, tools = [])
result = agent.run("Hello")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Hello                                                                                                           │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Hello! How can I assist you today?")                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Hello! How can I assist you today?

[Step 1: Duration 1.46 seconds| Input tokens: 1,937 | Output tokens: 71]

In [50]:
# define tools
@tool
def my_custom_tool (arg:str) -> str:
    """A dummy tool that does nothing
    Args:
        arg: dummy arg
    """
    return "Here is your magic tool that does nothing. "

In [ ]:
The CodeAgent assumes complex tasks might require multiple steps of reasoning and code execution:

Step 1: Analyze the problem and write initial code
Step 2: Examine results and decide if more work is needed
Step 3: Write additional code if necessary
Continue: Until the task is "complete"

In [70]:
# Agent with dummy tool
from smolagents.agents import EMPTY_PROMPT_TEMPLATES
prompt_templates = EMPTY_PROMPT_TEMPLATES.copy()

prompt_templates['system_prompt'] = """
        You are an assistant with access to a special tool called `my_custom_tool`.
        Whenever the user asks to "see magic", "show magic", "do magic", or otherwise refers
        to something magical, you MUST call `my_custom_tool` with the user’s provided phrase
        as the `arg` argument. 
        Thoughts: I need to call the magic tool
        <code>
        result = my_custom_tool(arg="user's request")
        print(result)
        </code>
        
        - If the user does not provide an argument, pass "magic" as the default.
        - Do not try to answer the request directly when it involves magic—always use the tool.
        - After calling the tool, return its output to the user.
        - For all other requests not related to magic, just answer normally without calling the tool.
        """ 

agent_dummy = CodeAgent(
    model= model, 
    tools = [my_custom_tool],
    prompt_templates = prompt_templates,
    max_steps = 1
)
result = agent_dummy.run("Show me some magic!")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Show me some magic!                                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-68b69480-0279ccbe00a2b2a80ef5f317;6e8ecdd7-c614-41a1-b5f2-21f2adc2f6b5)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.24 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-68b69480-0279ccbe00a2b2a80ef5f317;6e8ecdd7-c614-41a1-b5f2-21f2adc2f6b5)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [63]:
from datetime import datetime
import time

@tool
def get_time()->str:
    """A tool that fetches the current local time."""
    try:
        now = datetime.now()
        return f"🕒 It's {now.strftime('%I:%M %p')} on {now.strftime('%A, %B %d')}"
    except Exception as e:
        return f"❌ Error getting time: {str(e)}"
        
    
    

In [71]:
agent_time = CodeAgent(
    model= model, 
    tools = [get_time]
)
result = agent_time.run("What time?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What time?                                                                                                      │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-68b6948b-784318640fd693ca41bb34a2;f7d45ca7-f880-4035-bcf9-0c2061677081)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.24 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-68b6948b-784318640fd693ca41bb34a2;f7d45ca7-f880-4035-bcf9-0c2061677081)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.